In [2]:
import pandas as pd
import matplotlib.pyplot as plt  ## for plotting graph
import numpy as np   ## for numerical operation
import seaborn as sns ## for enhanced visualizations
from scipy.stats import mannwhitneyu, chi2_contingency, norm, skew  # For statistical tests
from statsmodels.stats.proportion import proportion_confint  # For proportion confidence intervals


In [3]:
# --- Data Loading ---
# Load datasets: dataset1.csv (individual bat landings) and dataset2.csv (30-minute aggregated observations)
dataset1 = pd.read_csv('dataset1.csv')
dataset2 = pd.read_csv('dataset2.csv')

In [28]:
print(dataset1.head())

##show all column
pd.set_option('display.max_columns', None)
#df.

           start_time  bat_landing_to_food habit    rat_period_start  \
0 2017-12-30 18:37:00            16.000000   rat 2017-12-30 18:35:00   
1 2017-12-30 19:51:00             0.074016  fast 2017-12-30 19:50:00   
2 2017-12-30 19:51:00             4.000000  fast 2017-12-30 19:50:00   
3 2017-12-30 19:52:00            10.000000   rat 2017-12-30 19:50:00   
4 2017-12-30 19:54:00            15.000000   rat 2017-12-30 19:50:00   

       rat_period_end  seconds_after_rat_arrival  risk  reward  month  \
0 2017-12-30 18:38:00                        108     1       0      0   
1 2017-12-30 19:55:00                         17     0       1      0   
2 2017-12-30 19:55:00                         41     0       1      0   
3 2017-12-30 19:55:00                        111     1       0      0   
4 2017-12-30 19:55:00                        194     1       0      0   

        sunset_time  hours_after_sunset  season         time_bucket  
0  30/12/2017 16:45            1.870833       0 2017-12-30

In [4]:
### Data Exploration Step for dat set 1
###############################
## check for missing Value,
##  Garbage/Invalid data 
## Distribution, Correlations, onte outlines

print("\n Dataset Inofromattion")
dataset1.info()   ## show data type and missing vlaue

print("\n Statistiacal Summary ")
print(dataset1.describe())  ## provides Summary stats

print("\nMissing Values:")
print(dataset1.isnull().sum())




 Dataset Inofromattion
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 907 entries, 0 to 906
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   start_time                 907 non-null    object 
 1   bat_landing_to_food        907 non-null    float64
 2   habit                      866 non-null    object 
 3   rat_period_start           907 non-null    object 
 4   rat_period_end             907 non-null    object 
 5   seconds_after_rat_arrival  907 non-null    int64  
 6   risk                       907 non-null    int64  
 7   reward                     907 non-null    int64  
 8   month                      907 non-null    int64  
 9   sunset_time                907 non-null    object 
 10  hours_after_sunset         907 non-null    float64
 11  season                     907 non-null    int64  
dtypes: float64(2), int64(5), object(5)
memory usage: 85.2+ KB

 Statistiacal Summary 


In [5]:
### Data Exploration Step for dat set 2
###############################
## check for missing Value,
##  Garbage/Invalid data 
## Distribution, Correlations, onte outlines

print("\n Dataset Inofromattion")
dataset2.info()   ## show data type and missing vlaue

print("\n Statistiacal Summary ")
print(dataset2.describe())  ## provides Summary stats

print("\nMissing Values:")
print(dataset2.isnull().sum())



 Dataset Inofromattion
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2123 entries, 0 to 2122
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   time                2123 non-null   object 
 1   month               2123 non-null   int64  
 2   hours_after_sunset  2123 non-null   float64
 3   bat_landing_number  2123 non-null   int64  
 4   food_availability   2123 non-null   float64
 5   rat_minutes         2123 non-null   float64
 6   rat_arrival_number  2123 non-null   int64  
dtypes: float64(3), int64(3), object(1)
memory usage: 116.2+ KB

 Statistiacal Summary 
             month  hours_after_sunset  bat_landing_number  food_availability  \
count  2123.000000         2123.000000         2123.000000        2123.000000   
mean      3.083844            5.265426           32.083373           2.445874   
std       1.642261            4.076188           25.614431           1.218353   
min       0.00000

In [6]:
print(dataset1.columns.tolist)

<bound method IndexOpsMixin.tolist of Index(['start_time', 'bat_landing_to_food', 'habit', 'rat_period_start',
       'rat_period_end', 'seconds_after_rat_arrival', 'risk', 'reward',
       'month', 'sunset_time', 'hours_after_sunset', 'season'],
      dtype='object')>


In [7]:
print(dataset2.columns.tolist)

<bound method IndexOpsMixin.tolist of Index(['time', 'month', 'hours_after_sunset', 'bat_landing_number',
       'food_availability', 'rat_minutes', 'rat_arrival_number'],
      dtype='object')>


In [8]:
### 41 data are missing form habit column
# 1. Handle Missing Values
# Check and address missing values in quantitative and qualitative variables to ensure data completeness
# Quantitative (dataset1): bat_landing_to_food, seconds_after_rat_arrival, hours_after_sunset
for col in ['bat_landing_to_food', 'seconds_after_rat_arrival', 'hours_after_sunset']:
    if dataset1[col].isnull().sum() > 0:
        print(f"Missing values in dataset1.{col}: {dataset1[col].isnull().sum()}")
        if col == 'bat_landing_to_food':
            # Drop rows with missing bat_landing_to_food, as it's critical for vigilance analysis
            dataset1 = dataset1.dropna(subset=[col])
        elif col == 'seconds_after_rat_arrival':
            # Missing seconds_after_rat_arrival indicates no rat was present; leave as is
            pass
        elif col == 'hours_after_sunset':
            # Impute with median of same month to preserve temporal patterns
            dataset1[col] = dataset1.groupby('month')[col].transform(lambda x: x.fillna(x.median()))



In [9]:
## checking for any messing  value data set 1
print(dataset1.isna().sum())

start_time                    0
bat_landing_to_food           0
habit                        41
rat_period_start              0
rat_period_end                0
seconds_after_rat_arrival     0
risk                          0
reward                        0
month                         0
sunset_time                   0
hours_after_sunset            0
season                        0
dtype: int64


In [10]:
# Quantitative (dataset2): bat_landing_number, food_availability, rat_minutes, rat_arrival_number, hours_after_sunset
for col in ['bat_landing_number', 'food_availability', 'rat_minutes', 'rat_arrival_number', 'hours_after_sunset']:
    if dataset2[col].isnull().sum() > 0:
        print(f"Missing values in dataset2.{col}: {dataset2[col].isnull().sum()}")
        if col == 'food_availability':
            # Impute with median of same month to account for seasonal patterns
            dataset2[col] = dataset2.groupby('month')[col].transform(lambda x: x.fillna(x.median()))
        elif col in ['rat_minutes', 'rat_arrival_number']:
            # Assume 0 for missing rat presence (no rats in that period)
            dataset2[col] = dataset2[col].fillna(0)
        else:
            # Drop rows with missing bat_landing_number or hours_after_sunset
            dataset2 = dataset2.dropna(subset=[col])

In [11]:
# 2. Correct Data Types
# Ensure columns have appropriate types for analysis and merging
try:
    # Convert time columns to datetime for temporal alignment
    dataset1['start_time'] = pd.to_datetime(dataset1['start_time'], format='%d/%m/%Y %H:%M', errors='coerce')
    dataset2['time'] = pd.to_datetime(dataset2['time'], format='%d/%m/%Y %H:%M', errors='coerce')
except Exception as e:
    print(f"Error converting datetime: {e}")


In [12]:
# Convert numeric columns to float/int, handling invalid entries
numeric_cols_d1 = ['bat_landing_to_food', 'seconds_after_rat_arrival', 'hours_after_sunset']
numeric_cols_d2 = ['bat_landing_number', 'food_availability', 'rat_minutes', 'rat_arrival_number', 'hours_after_sunset']
for col in numeric_cols_d1:
    dataset1[col] = pd.to_numeric(dataset1[col], errors='coerce')
for col in numeric_cols_d2:
    dataset2[col] = pd.to_numeric(dataset2[col], errors='coerce')


In [13]:
# Ensure binary (risk, reward) and categorical (habit) columns are correctly formatted
dataset1['risk'] = dataset1['risk'].astype(int, errors='ignore')
dataset1['reward'] = dataset1['reward'].astype(int, errors='ignore')
dataset1['habit'] = dataset1['habit'].astype(str).str.lower()  # Standardize to lowercase for consistency

In [14]:
# 3. Handle Outliers
# Use IQR method to cap outliers in quantitative variables to prevent skewing results
def cap_outliers(df, col):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df[col] = df[col].clip(lower=lower_bound, upper=upper_bound)
    return df


In [15]:
# Apply outlier capping to quantitative columns (except seconds_after_rat_arrival, which is context-specific)
for col in numeric_cols_d1:
    if col != 'seconds_after_rat_arrival':
        dataset1 = cap_outliers(dataset1, col)
for col in numeric_cols_d2:
    dataset2 = cap_outliers(dataset2, col)


In [16]:
# 4. Remove Duplicates
# Remove duplicate rows to avoid double-counting events or periods
dataset1 = dataset1.drop_duplicates()
dataset2 = dataset2.drop_duplicates()

In [26]:
# 5. Validate Temporal Consistency
# Ensure seconds_after_rat_arrival aligns with rat_period_start and rat_period_end
dataset1['rat_period_start'] = pd.to_datetime(dataset1['rat_period_start'], errors='coerce')
dataset1['rat_period_end'] = pd.to_datetime(dataset1['rat_period_end'], errors='coerce')
invalid_time = dataset1[
    (dataset1['seconds_after_rat_arrival'].notnull()) &
    (~dataset1['start_time'].between(dataset1['rat_period_start'], dataset1['rat_period_end']))
]
if not invalid_time.empty:
    print(f"Found {len(invalid_time)} inconsistent time entries; dropping them.")
    dataset1 = dataset1.drop(invalid_time.index)


In [18]:

# 6. Standardize habit categories
# Fix known inconsistencies in habit (e.g., typos or case variations)
dataset1['habit'] = dataset1['habit'].replace({'Fast': 'fast', 'Pick': 'pick'})

In [19]:
# 7. Merge Datasets
# Align dataset1's start_time to dataset2's 30-minute time buckets for merging
dataset1['time_bucket'] = dataset1['start_time'].dt.floor('30min')
merged_data = pd.merge(dataset1, dataset2, left_on='time_bucket', right_on='time', how='left')

In [20]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 906 entries, 0 to 905
Data columns (total 20 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   start_time                 906 non-null    datetime64[ns]
 1   bat_landing_to_food        906 non-null    float64       
 2   habit                      906 non-null    object        
 3   rat_period_start           906 non-null    datetime64[ns]
 4   rat_period_end             906 non-null    datetime64[ns]
 5   seconds_after_rat_arrival  906 non-null    int64         
 6   risk                       906 non-null    int64         
 7   reward                     906 non-null    int64         
 8   month_x                    906 non-null    int64         
 9   sunset_time                906 non-null    object        
 10  hours_after_sunset_x       906 non-null    float64       
 11  season                     906 non-null    int64         
 12  time_buc

In [27]:
# Handle unmatched rows in merged_data (e.g., missing food_availability)
if merged_data['food_availability'].isnull().sum() > 0:
    print(f"Missing food_availability after merge: {merged_data['food_availability'].isnull().sum()}")
    merged_data['food_availability'] = merged_data.groupby('month')['food_availability'].transform(lambda x: x.fillna(x.median()))

Missing food_availability after merge: 903


KeyError: 'month'

In [ ]:
# 8. Feature Engineering
# Create a binary rat_present column to simplify analysis (1 if rat was present, 0 otherwise)
merged_data['rat_present'] = merged_data['seconds_after_rat_arrival'].notnull() | (merged_data['rat_minutes'] > 0)

In [ ]:
# Define rat-present and rat-absent groups for analysis
rat_present = merged_data[merged_data['rat_present']]
rat_absent = merged_data[~merged_data['rat_present']]


In [ ]:
# --- Descriptive Analysis: Quantitative Variables ---
# Define quantitative variables to analyze from both datasets
quant_vars = {
    'dataset1': ['bat_landing_to_food', 'seconds_after_rat_arrival', 'hours_after_sunset'],
    'dataset2': ['bat_landing_number', 'food_availability', 'rat_minutes', 'rat_arrival_number']
   }


In [ ]:
 ##Loop through datasets and variables to compute descriptive statistics
for dataset_name, vars_list in quant_vars.items():
    data = merged_data if dataset_name == 'dataset1' else dataset2
    print(f"\nDescriptive Analysis for {dataset_name}:")
    for var in vars_list:
        # Use rat_present for seconds_after_rat_arrival, as it's only relevant when rats are present
        if var == 'seconds_after_rat_arrival':
            values = rat_present[var].dropna()
        else:
            values = data[var].dropna()
        
        # Compute measures of center: mean, median, mode
        mean_val = values.mean()
        median_val = values.median()
        mode_val = values.mode()[0] if not values.mode().empty else np.nan
        
        # Compute dispersion: std, range, variance, IQR
        std_val = values.std()
        range_val = values.max() - values.min()
        variance_val = values.var()
        iqr_val = np.percentile(values, 75) - np.percentile(values, 25)
        
        # Compute shape: skewness to determine if symmetric, right-skewed, or left-skewed
        skewness = skew(values, nan_policy='omit')
        shape = 'Symmetric' if abs(skewness) < 0.5 else 'Right-skewed' if skewness > 0.5 else 'Left-skewed'
        
        # Print statistics for the variable
        print(f"\n{var}:")
        print(f"  Mean: {mean_val:.2f}, Median: {median_val:.2f}, Mode: {mode_val:.2f}")
        print(f"  Std Dev: {std_val:.2f}, Range: {range_val:.2f}, Variance: {variance_val:.2f}, IQR: {iqr_val:.2f}")
        print(f"  Skewness: {skewness:.2f} ({shape})")
        
        # Visualize distribution with histogram (with KDE) and boxplot
        plt.figure(figsize=(10, 4))
        plt.subplot(1, 2, 1)
        sns.histplot(values, kde=True)
        plt.title(f'Histogram of {var}')
        plt.subplot(1, 2, 2)
        sns.boxplot(y=values)
        plt.title(f'Boxplot of {var}')
        plt.tight_layout()
        plt.show()




In [2]:
# --- Descriptive Analysis: Qualitative Variables ---
# Analyze categorical/binary variables: risk, reward, habit
qual_vars = ['risk', 'reward', 'habit']
for var in qual_vars:
    print(f"\nFrequency Table for {var}:")
    # Compute frequency table for each category, including NaNs if any
    freq_table = merged_data[var].value_counts(dropna=False)
    print(freq_table)
    
    # Bar chart to show counts of each category
    plt.figure(figsize=(8, 4))
    sns.countplot(x=var, data=merged_data)
    plt.title(f'Bar Chart of {var}')
    plt.show()
    
    # Pie chart to show proportion of each category
    plt.figure(figsize=(6, 6))
    freq_table.plot.pie(autopct='%1.1f%%')
    plt.title(f'Pie Chart of {var}')
    plt.ylabel('')
    plt.show()

In [3]:
# --- Inferential Statistics ---
# Define function to compute 95% confidence interval for mean
def compute_ci_mean(data, confidence=0.95):
    mean = data.mean()
    std = data.std()
    n = len(data)
    se = std / np.sqrt(n)  # Standard error
    ci = norm.ppf(1 - (1 - confidence) / 2) * se  # Z-value for 95% CI
    return mean, (mean - ci, mean + ci)

In [ ]:
# Compute 95% CI for bat_landing_to_food in rat-present and rat-absent groups
mean_present, ci_present = compute_ci_mean(rat_present['bat_landing_to_food'].dropna())
mean_absent, ci_absent = compute_ci_mean(rat_absent['bat_landing_to_food'].dropna())
print(f"\n95% CI for bat_landing_to_food (Rat Present): {mean_present:.2f}, {ci_present}")
print(f"95% CI for bat_landing_to_food (Rat Absent): {mean_absent:.2f}, {ci_absent}")


In [ ]:
# Compute 95% CI for proportion of reward=1
count_present = rat_present['reward'].sum()
n_present = len(rat_present)
count_absent = rat_absent['reward'].sum()
n_absent = len(rat_absent)
ci_reward_present = proportion_confint(count_present, n_present, alpha=0.05, method='normal')
ci_reward_absent = proportion_confint(count_absent, n_absent, alpha=0.05, method='normal')
print(f"95% CI for Reward Proportion (Rat Present): {count_present/n_present:.2f}, {ci_reward_present}")
print(f"95% CI for Reward Proportion (Rat Absent): {count_absent/n_absent:.2f}, {ci_reward_absent}")

In [ ]:
# Hypothesis Tests
# H1: Bats take longer to approach food when rats are present (bat_landing_to_food higher)
# Use Mann-Whitney U test (non-parametric) due to likely skewed time data
u_stat, p_val = mannwhitneyu(rat_present['bat_landing_to_food'].dropna(), 
                             rat_absent['bat_landing_to_food'].dropna(), alternative='two-sided')
print(f"\nMann-Whitney U Test for bat_landing_to_food (p-value): {p_val:.4f}")


In [ ]:
# H2: Bats are less likely to gain rewards when rats are present
# Use chi-square test to compare reward proportions
contingency_table = pd.crosstab(merged_data['rat_present'], merged_data['reward'])
chi2, p_chi, _, _ = chi2_contingency(contingency_table)
print(f"Chi-square Test for reward (p-value): {p_chi:.4f}")

In [ ]:
# H3: Bat landings decrease with increased rat presence
# Use Spearman correlation (non-parametric) to test relationship between bat_landing_number and rat_minutes
corr, p_corr = dataset2[['bat_landing_number', 'rat_minutes']].corr(method='spearman').iloc[0, 1], \
               dataset2[['bat_landing_number', 'rat_minutes']].corr(method='spearman', min_periods=1).iloc[0, 1]
print(f"Spearman Correlation (bat_landing_number vs rat_minutes): {corr:.4f}, p-value: {p_corr:.4f}")

In [ ]:
# --- Scatter Plots ---
# Scatter plot: seconds_after_rat_arrival vs bat_landing_to_food
# Tests if vigilance (time to food) increases with recency of rat arrival
plt.figure(figsize=(8, 6))
sns.scatterplot(x='seconds_after_rat_arrival', y='bat_landing_to_food', data=rat_present)
plt.title('bat_landing_to_food vs seconds_after_rat_arrival')
plt.xlabel('Seconds After Rat Arrival')
plt.ylabel('Bat Landing to Food (seconds)')
plt.show()

In [ ]:
# --- Scatter Plots ---
# Scatter plot: seconds_after_rat_arrival vs bat_landing_to_food
# Tests if vigilance (time to food) increases with recency of rat arrival
plt.figure(figsize=(8, 6))
sns.scatterplot(x='seconds_after_rat_arrival', y='bat_landing_to_food', data=rat_present)
plt.title('bat_landing_to_food vs seconds_after_rat_arrival')
plt.xlabel('Seconds After Rat Arrival')
plt.ylabel('Bat Landing to Food (seconds)')
plt.show()

In [ ]:
# Scatter plot: food_availability vs bat_landing_number
# Tests if food levels influence bat foraging, potentially moderated by rats
plt.figure(figsize=(8, 6))
sns.scatterplot(x='food_availability', y='bat_landing_number', data=dataset2)
plt.title('bat_landing_number vs food_availability')
plt.xlabel('Food Availability')
plt.ylabel('Bat Landing Number')
plt.show()

In [ ]:
# --- Conclusion: Hypothesis Testing and Interpretation ---
# Hypotheses:
# H1: Bats take longer to approach food when rats are present (higher bat_landing_to_food)
# H2: Bats are less likely to gain rewards when rats are present (lower reward proportion)
# H3: Bat landings decrease with increased rat presence (negative correlation with rat_minutes)
print("\nConclusion for Investigation A: Do bats perceive rats as potential predators?\n")

# H1: Interpret Mann-Whitney U test for bat_landing_to_food
if p_val < 0.05:
    print("H1: Supported. Bats take significantly longer to approach food when rats are present (p = {:.4f}).".format(p_val))
    print("  Mean bat_landing_to_food (Rat Present): {:.2f}, (Rat Absent): {:.2f}".format(mean_present, mean_absent))
    print("  This suggests increased vigilance, consistent with perceiving rats as predators.")
else:
    print("H1: Not supported. No significant difference in bat_landing_to_food between rat-present and rat-absent conditions (p = {:.4f}).".format(p_val))
    print("  Mean bat_landing_to_food (Rat Present): {:.2f}, (Rat Absent): {:.2f}".format(mean_present, mean_absent))
    print("  Bats may not exhibit increased vigilance due to rats.")

# H2: Interpret chi-square test for reward
if p_chi < 0.05:
    print("H2: Supported. Bats are significantly less likely to gain rewards when rats are present (p = {:.4f}).".format(p_chi))
    print("  Reward proportion (Rat Present): {:.2f}, (Rat Absent): {:.2f}".format(count_present/n_present, count_absent/n_absent))
    print("  This suggests rats may be perceived as a threat, reducing foraging success.")
else:
    print("H2: Not supported. No significant difference in reward proportion between rat-present and rat-absent conditions (p = {:.4f}).".format(p_chi))
    print("  Reward proportion (Rat Present): {:.2f}, (Rat Absent): {:.2f}".format(count_present/n_present, count_absent/n_absent))
    print("  Rats may primarily act as competitors rather than predators.")

# H3: Interpret Spearman correlation for bat_landing_number vs rat_minutes
if p_corr < 0.05 and corr < 0:
    print("H3: Supported. Significant negative correlation between bat_landing_number and rat_minutes (corr = {:.4f}, p = {:.4f}).".format(corr, p_corr))
    print("  Increased rat presence is associated with fewer bat landings, suggesting avoidance behavior.")
else:
    print("H3: Not supported. No significant negative correlation between bat_landing_number and rat_minutes (corr = {:.4f}, p = {:.4f}).".format(corr, p_corr))
    print("  Bat foraging activity may not be strongly affected by rat presence.")


In [ ]:
# Overall interpretation
print("\nOverall Interpretation:")
if p_val < 0.05 or (p_chi < 0.05 and count_present/n_present < count_absent/n_absent) or (p_corr < 0.05 and corr < 0):
    print("The data supports the hypothesis that bats perceive rats as potential predators, not just competitors.")
    print("Evidence includes:")
    if p_val < 0.05:
        print("- Increased time to approach food (bat_landing_to_food) when rats are present, indicating higher vigilance.")
    if p_chi < 0.05 and count_present/n_present < count_absent/n_absent:
        print("- Lower reward rates in rat-present conditions, suggesting fear or competition reduces foraging success.")
    if p_corr < 0.05 and corr < 0:
        print("- Fewer bat landings when rats are present longer, indicating avoidance behavior.")
else:
    print("The data does not strongly support the hypothesis that bats perceive rats as predators.")
    print("Bats may primarily view rats as competitors for food, as evidenced by:")
    print("- No significant increase in bat_landing_to_food (p = {:.4f}).".format(p_val))
    print("- Similar reward rates (p = {:.4f}).".format(p_chi))
    print("- No strong negative correlation between bat landings and rat presence (p = {:.4f}).".format(p_corr))
print("Further analysis (e.g., logistic regression on risk-taking) or additional data (e.g., direct predation events) could clarify the extent of predator perception.")